In [ ]:
import cv2
import numpy as np


def add_bottom_overlay(image, percent=20, alpha=0.5):
    height, width, _ = image.shape
    overlay_height = int(height * (percent / 100))
    overlay = image.copy()
    cv2.rectangle(overlay, (0, height - overlay_height), (width, height), (0, 0, 0), -1)
    image[height - overlay_height : height, 0:width] = cv2.addWeighted(
        overlay[height - overlay_height : height, 0:width],
        alpha,
        image[height - overlay_height : height, 0:width],
        1 - alpha,
        0,
    )


def draw_text(
    image,
    text,
    position,
    font=cv2.FONT_HERSHEY_SIMPLEX,
    font_scale=0.8,
    color=(255, 255, 255),
    thickness=2,
):
    """Draw text on an image."""
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)


def draw_bar(
    image,
    position,
    width,
    height,
    level,
    label,
    bar_bg_color=(100, 100, 100),
    bar_fg_color_start=(255, 255, 255),
    bar_fg_color_end=(150, 150, 150),
):
    """Draw a bar with a gradient and rounded corners on an image."""

    def draw_rounded_rect(img, pt1, pt2, color, radius):
        x1, y1 = pt1
        x2, y2 = pt2
        cv2.rectangle(img, (x1 + radius, y1), (x2 - radius, y2), color, -1)
        cv2.rectangle(img, (x1, y1 + radius), (x2, y2 - radius), color, -1)
        cv2.ellipse(
            img, (x1 + radius, y1 + radius), (radius, radius), 180, 0, 90, color, -1
        )
        cv2.ellipse(
            img, (x2 - radius, y1 + radius), (radius, radius), 270, 0, 90, color, -1
        )
        cv2.ellipse(
            img, (x1 + radius, y2 - radius), (radius, radius), 90, 0, 90, color, -1
        )
        cv2.ellipse(
            img, (x2 - radius, y2 - radius), (radius, radius), 0, 0, 90, color, -1
        )

    def draw_gradient_bar(img, pt1, pt2, start_color, end_color):
        x1, y1 = pt1
        x2, y2 = pt2
        bar_width = x2 - x1
        bar_height = y2 - y1

        gradient_bar = np.zeros((bar_height, bar_width, 3), dtype=np.uint8)
        for i in range(bar_width):
            alpha = i / bar_width
            color = tuple(
                int(start_color[j] * (1 - alpha) + end_color[j] * alpha)
                for j in range(3)
            )
            cv2.line(gradient_bar, (i, 0), (i, bar_height), color, 1)

        rounded_bar = np.zeros((bar_height, bar_width, 3), dtype=np.uint8)
        draw_rounded_rect(
            rounded_bar,
            (0, 0),
            (bar_width, bar_height),
            (255, 255, 255),
            bar_height // 2,
        )

        mask = cv2.cvtColor(rounded_bar, cv2.COLOR_BGR2GRAY)
        mask_inv = cv2.bitwise_not(mask)

        img_bg = cv2.bitwise_and(img[y1:y2, x1:x2], img[y1:y2, x1:x2], mask=mask_inv)
        bar_fg = cv2.bitwise_and(gradient_bar, gradient_bar, mask=mask)

        img[y1:y2, x1:x2] = cv2.add(img_bg, bar_fg)

    x, y = position
    bar_level = int(width * level)

    # Bar background and foreground
    draw_rounded_rect(image, (x, y), (x + width, y + height), bar_bg_color, height // 2)
    draw_gradient_bar(
        image, (x, y), (x + bar_level, y + height), bar_fg_color_start, bar_fg_color_end
    )
    draw_text(image, label, (x + width + 10, y + 15))


def draw_circ_bool(img, state, pos, rad=12, color=(255, 255, 255)):
    thickness = cv2.FILLED if state else 1
    cv2.circle(img, pos, rad, color, thickness, cv2.LINE_AA)


def draw_battery(
    image,
    position,
    width,
    height,
    level,
    border_color=(255, 255, 255),
    fill_color=(207, 255, 207),
    background_color=(20, 20, 20),
):
    """
    Draw a battery icon with the specified fill level.

    Parameters:
    - image: The image on which to draw the battery.
    - position: A tuple (x, y) representing the top-left corner of the battery icon.
    - width: The width of the battery icon.
    - height: The height of the battery icon.
    - level: The fill level of the battery (0 to 1).
    - border_color: The color of the battery border.
    - fill_color: The color of the battery fill.
    - background_color: The color of the battery background.
    """
    x, y = position
    # Draw the main battery rectangle
    cv2.rectangle(image, (x, y), (x + width, y + height), border_color, 2)

    # Draw the positive terminal
    terminal_width = int(width * 0.08)
    cv2.rectangle(
        image,
        (x + width, y + int(height * 0.3)),
        (x + width + terminal_width, y + int(height * 0.7)),
        border_color,
        -1,
    )

    # Draw the battery background
    cv2.rectangle(
        image, (x + 2, y + 2), (x + width - 2, y + height - 2), background_color, -1
    )

    # Draw the filled part of the battery
    fill_width = int((width - 4) * level)
    cv2.rectangle(
        image, (x + 2, y + 2), (x + 2 + fill_width, y + height - 2), fill_color, -1
    )


import cv2
import numpy as np


def draw_line_graph(
    image,
    data,
    position,
    graph_size,
    axis_color=(255, 255, 255),
    line_color=(0, 255, 255),
    thickness=2,
):
    """
    Draw a line graph on an image.

    Parameters:
    - image: The image on which to draw the graph.
    - data: A list of normalized data points (values between -1 and 1).
    - position: A tuple (x, y) representing the top-left corner of the graph.
    - graph_size: A tuple (width, height) representing the size of the graph.
    - axis_color: The color of the graph axes.
    - line_color: The color of the graph line.
    - thickness: The thickness of the graph line.
    """
    x, y = position
    width, height = graph_size

    # Draw horizontal axis
    cv2.line(
        image, (x, y + height // 2), (x + width, y + height // 2), axis_color, thickness
    )

    # Draw graph line
    num_points = len(data)
    step = width // (num_points - 1)

    for i in range(1, num_points):
        pt1 = (x + (i - 1) * step, y + height // 2 - int(data[i - 1] * (height // 2)))
        pt2 = (x + i * step, y + height // 2 - int(data[i] * (height // 2)))
        cv2.line(image, pt1, pt2, line_color, thickness)

    # Draw vertical axis
    cv2.line(image, (x, y), (x, y + height), axis_color, thickness)

In [ ]:
import numpy as np
import cv2


def draw_axes(frame, angles, center=(320, 240), length=100):
    """
    Draws 3D axes on the given frame.

    Parameters:
    frame: The image frame on which to draw the axes.
    angles: A tuple or list of three angles (roll, pitch, yaw) in degrees.
    center: The center point for the axes (default is (320, 240) for a 640x480 frame).
    length: The length of each axis line (default is 100).
    """
    roll, pitch, yaw = np.deg2rad(angles)

    # Rotation matrices
    Rx = np.array(
        [[1, 0, 0], [0, np.cos(roll), -np.sin(roll)], [0, np.sin(roll), np.cos(roll)]]
    )

    Ry = np.array(
        [
            [np.cos(pitch), 0, np.sin(pitch)],
            [0, 1, 0],
            [-np.sin(pitch), 0, np.cos(pitch)],
        ]
    )

    Rz = np.array(
        [[np.cos(yaw), -np.sin(yaw), 0], [np.sin(yaw), np.cos(yaw), 0], [0, 0, 1]]
    )

    # Combined rotation matrix
    R = Rz @ Ry @ Rx

    # Define the axes in 3D space
    axes = np.array(
        [
            [length, 0, 0],  # X-axis (red)
            [0, length, 0],  # Y-axis (green)
            [0, 0, length],  # Z-axis (blue)
        ]
    )

    # Project the 3D axes to 2D
    axes_2d = np.dot(axes, R.T).astype(int)

    # Define colors for the axes
    colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)]

    for axis, color in zip(axes_2d, colors):
        pt = (center[0] + axis[0], center[1] - axis[1])
        cv2.line(frame, center, pt, color, 2, cv2.LINE_AA)

    return frame


import pandas as pd

data = {
    "roll": np.linspace(0, 360, 100),
    "pitch": np.sin(np.linspace(0, 2 * np.pi, 100)) * 45,
    "yaw": np.cos(np.linspace(0, 2 * np.pi, 100)) * 45,
}
df = pd.DataFrame(data)

In [ ]:
import vidformer
import cv2

server = vidformer.YrdenServer(bin="../target/release/vidformer-cli")
# server = vidformer.YrdenServer(domain='localhost', port=5000)

scale = vidformer.Filter("Scale")


class MyFilter(vidformer.UDF):
    def filter(self, frame: vidformer.UDFFrame, i: int):
        f = frame.data().copy()
        height, width = f.shape[:2]
        add_bottom_overlay(f, percent=25)

        speed, altitude, lox, ch4 = (
            3808 + i // 3,
            43 + i // 34,
            max(0.8 - i * 0.001, 0.1),
            max(1.0 - i * 0.003, 0.1),
        )
        draw_text(f, f"SPEED       {speed} KM/H", (200, height - 150))
        draw_text(f, f"ALTITUDE    {altitude} KM", (200, height - 110))

        # Draw the LOX and CH4 bars
        draw_bar(f, (200, height - 40), 200, 20, lox, "LOX")
        draw_bar(f, (200, height - 80), 200, 20, ch4, "CH4")

        draw_text(f, f"T+{i/24:0.2f}", (575, height - 75), font_scale=2.0, thickness=4)

        draw_text(f, f"ALPHA", (945, height - 90))
        draw_circ_bool(f, False, (920, height - 100))

        draw_text(f, f"GAMMA", (945, height - 50))
        draw_circ_bool(f, i % 50 < 25, (920, height - 60))

        # draw_battery(f, (50, 50), 100, 40, 0.75)

        import math

        num_points = 50
        data = [
            0.6 * math.sin(2 * math.pi * 2 * x / num_points + i / 25)
            for x in range(num_points)
        ]
        position = (1080, height - 150)  # (x, y)
        size = (150, 100)  # (width, height)
        # Draw the line graph on the image
        draw_line_graph(f, data, position, size)

        draw_axes(f, df.iloc[i % len(df)].values, center=(100, height - 100), length=80)

        return vidformer.UDFFrame(f, frame.frame_type())

    def filter_type(self, frame, text):
        return frame


mf_udf = MyFilter("MyFilter")
mf = mf_udf.into_filter()

tos = vidformer.Source(server, "tos_720p", "tos_720p.mp4", 0)

domain = tos.ts()


def render(t, i):
    return scale(
        mf(scale(tos[t], format="rgb24", width=1280, height=720), i),
        format="yuv420p",
        width=1280,
        height=720,
    )


spec = vidformer.Spec(domain, render, tos.fmt())
# spec.play(server)
spec.save(server, "tos-hud.mp4")